## 单流体嵌入网络的PFGs聚合物

In [41]:
from time import time
from datetime import datetime
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import random, genfromtxt
from IPython.display import display
from matplotlib import rc
from matplotlib.pyplot import figure
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F
import matplotlib.ticker as mticker
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score

In [ ]:
current_directory = os.getcwd()
print(current_directory)
device=torch.device("cuda")
# 检查是否有可用的 GPU
if torch.cuda.is_available():
    device = torch.device("cuda")  # 使用 GPU
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")  # 使用 CPU
    print("No GPU available, using CPU instead.")

In [ ]:
# 设置数据类型
DTYPE = torch.float32

# 设置随机种子
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
random.seed(SEED)

df = {}
# 读取 Excel 文件中的所有 sheet
url_hf = 'Data/DifferComponent_data.xlsm'
df_HF = pd.read_excel(url_hf, sheet_name=None)
# 初始化训练集和验证集
train_set = pd.DataFrame()  # 用于存储训练集
valid_set = pd.DataFrame()  # 用于存储验证集

# 遍历所有 sheet
for sheet_name, df in df_HF.items():
    # 移除包含 NaN 的行
    df = df.dropna()
    
    # 根据 sheet_name 划分数据集
    if sheet_name == 'b2': 
        valid_set = df  # sheet_name='b1' 的视为验证集
    if sheet_name!='b1':
        train_set = pd.concat([train_set, df], ignore_index=True)  # 其他 sheet 拼接为训练集

# 提取目标变量
y1_train = train_set['lossF'].to_numpy()
y1_valid = valid_set['lossF'].to_numpy()

# 提取特征变量
feature_columns = ['AngFreq', 'Mn1', 'Mn2', 'Mn3',  
                   'Mn11', 'Mn22', 'Mn33']
X_train = train_set[feature_columns].to_numpy()
X_valid = valid_set[feature_columns].to_numpy()

# 计算哈达玛积
def add_hadamard_features(X):
    """
    计算哈达玛积并拼接新特征。

    参数:
        X (numpy.ndarray): 输入的特征矩阵，形状为 (n_samples, n_features)。
                          列顺序必须为 ['AngFreq', 'Mn1', 'Mn2', 'Mn3', 'Mn11', 'Mn22', 'Mn33']。

    返回:
        numpy.ndarray: 处理后的特征矩阵，形状为 (n_samples, 4)。
                      列顺序为 ['AngFreq', 'Mn1*Mn11', 'Mn2*Mn22', 'Mn3*Mn33']。
    """
   # 计算哈达玛积
    Mn1_Mn11 = X[:, 1] * X[:, 4]  # Mn1 * Mn11
    Mn2_Mn22 = X[:, 2] * X[:, 5]  # Mn2 * Mn22
    Mn3_Mn33 = X[:, 3] * X[:, 6]  # Mn3 * Mn33

    # 计算交叉相乘
    feature_1_2 = Mn1_Mn11 * Mn2_Mn22  # (Mn1 * Mn11) * (Mn2 * Mn22)
    feature_1_3 = Mn1_Mn11 * Mn3_Mn33  # (Mn1 * Mn11) * (Mn3 * Mn33)
    feature_2_3 = Mn2_Mn22 * Mn3_Mn33  # (Mn2 * Mn22) * (Mn3 * Mn33)
    feature_1_2_3=Mn1_Mn11*Mn2_Mn22*Mn3_Mn33
    # 拼接新特征
    X_new = np.column_stack((
        X[:, 0],  # AngFreq
        Mn1_Mn11,
        Mn2_Mn22,
        Mn3_Mn33,
        feature_1_2,
        feature_1_3,
        feature_2_3
    ))
    return X_new
X_train=add_hadamard_features(X_train)
X_valid=add_hadamard_features(X_valid)
# 对频率特征进行对数化处理
X_train[:, 0] = np.log10(X_train[:, 0])  # AngFreq 是第一个特征
X_valid[:, 0] = np.log10(X_valid[:, 0])
# 初始化 MinMaxScaler
x_scaler = MinMaxScaler()  # 用于特征 X 的归一化
y_scaler = MinMaxScaler()  # 用于目标 y 的归一化

# 对 X_train 进行归一化
X_train_normalized = x_scaler.fit_transform(X_train)

# 对 y_train 进行归一化
# 注意：y_train 需要 reshape 为二维数组，因为 MinMaxScaler 接受二维输入
y_train_normalized = y_scaler.fit_transform(y1_train.reshape(-1, 1))

# 使用保存的归一化参数对 X_valid 进行归一化
X_valid_normalized = x_scaler.transform(X_valid)
# 使用保存的归一化参数对 y_valid 进行归一化
y_valid_normalized = y_scaler.transform(y1_valid.reshape(-1, 1))

# 将归一化后的数据转换为 PyTorch 张量
X_data_HF = torch.tensor(X_train_normalized, dtype=torch.float32)
y_data_HF = torch.tensor(y_train_normalized, dtype=torch.float32)
X_data_valid = torch.tensor(X_valid_normalized, dtype=torch.float32)
y_data_valid = torch.tensor(y_valid_normalized, dtype=torch.float32)

# 打印张量的形状以确认
print("X_data_HF shape:", X_data_HF.shape)
print("y_data_HF shape:", y_data_HF.shape)
print("X_data_valid shape:", X_data_valid.shape)
print("y_data_valid shape:", y_data_valid.shape)

# 定义模型
in_dim, out_dim = 7, 1

In [44]:
# 神经网络的类
class PINN_NeuralNet(nn.Module):
    """ Set basic architecture of the PINN model."""

    def __init__(self,
                 input_dim=0,
                 output_dim=1,  # 默认输出维度为1
                 num_hidden_layers=4, 
                 num_neurons_per_layer=20,
                 activation='tanh',
                 kernel_initializer='glorot_normal',
                 **kwargs):
        super(PINN_NeuralNet, self).__init__()

        self.num_hidden_layers = num_hidden_layers
        self.output_dim = output_dim
        
        # 添加输入层
        self.input_layer = nn.Linear(input_dim, num_neurons_per_layer)
        
        # 添加其他隐藏层
        self.hidden_layers = nn.ModuleList()
        for _ in range(num_hidden_layers):
            self.hidden_layers.append(nn.Linear(num_neurons_per_layer, num_neurons_per_layer))
        
        # 添加输出层
        self.out = nn.Linear(num_neurons_per_layer, output_dim)
        # 设置激活函数
        if activation == 'tanh':
            self.activation = torch.tanh
        elif activation == 'relu':
            self.activation = F.relu6
        elif activation == 'sigmoid':
            self.activation = torch.sigmoid
        elif activation == 'linear':
            self.activation = None
        else:
            raise ValueError("Unsupported activation function")
        
        # 初始化权重
        if kernel_initializer == 'glorot_normal':
            nn.init.xavier_normal_(self.input_layer.weight)
            for hidden_layer in self.hidden_layers:
                nn.init.xavier_normal_(hidden_layer.weight)
            nn.init.xavier_normal_(self.out.weight)
        elif kernel_initializer == 'glorot_uniform':
            nn.init.xavier_uniform_(self.input_layer.weight)
            for hidden_layer in self.hidden_layers:
                nn.init.xavier_uniform_(hidden_layer.weight)
            nn.init.xavier_uniform_(self.out.weight)
        else:
            raise ValueError("Unsupported kernel initializer")

    def forward(self, X):

        # 进入输入层
        Z = self.input_layer(X)
        
        # 通过隐藏层
        for hidden_layer in self.hidden_layers:
            Z = hidden_layer(Z)
            if self.activation is not None:
                Z = self.activation(Z)
        # 通过输出层输出
        Z = self.out(Z)
        
        return Z

In [45]:
# 神经网络处理器类
class PINNSolver():

    # 类属性定义
    def __init__(self, model_HF_nl,model_HF_1):
        
        self.model_HF_nl = model_HF_nl # 高保真非线性模型
        self.model_HF_l=model_HF_1
        self.hist =  [[], []] # loss历史列表,0:train loss ;1:valid loss
        self.iter = 0 # 迭代次数
        self.last_n_losses = [] # 前损失列表

    # 更新损失列表   
    def update_last_n_losses(self, loss):
        self.last_n_losses.append(loss)
        if len(self.last_n_losses) > 20:
            self.last_n_losses.pop(0)

    # 计算最大相对误差        
    def ES(self):
        if len(self.last_n_losses) < 20:
            return 100  # a large number

        current_loss = self.last_n_losses[-1]
        max_relative_error = 100.*max([abs(current_loss - loss) / current_loss for loss in self.last_n_losses[:-1]])
        return max_relative_error
    
    # 计算loss，模型核心
    def loss_fn(self, X_data_HF, y_data_HF,X_data_valid,y_data_valid):
        y_pred_valid_nl = self.model_HF_nl(X_data_valid)
        y_pred_valid_l = self.model_HF_l(X_data_valid)
        y_pred_valid = y_pred_valid_nl+y_pred_valid_l    
        y_pred_HF_nl = self.model_HF_nl(X_data_HF)
        y_pred_HF_l = self.model_HF_l(X_data_HF)
        y_pred_HF = y_pred_HF_nl+y_pred_HF_l
        
        Loss_L2 = 1e-5 * sum(torch.sum(w_**2) for w_ in self.model_HF_nl.parameters())
        Loss_L2+=1e-5*sum(torch.sum(w_**2) for w_ in self.model_HF_l.parameters())
        Loss_data_HF = torch.mean((y_data_HF - y_pred_HF)**2)+Loss_L2
        Loss_data_valid=torch.mean((y_pred_valid-y_data_valid)**2)+Loss_L2
        
        return Loss_data_HF,Loss_data_valid
    # 训练核心函数，包括loss计算梯度计算和反向传播
    def solve_with_PyTorch_optimizer(self, optimizer,data,scheduler,N=1001):
        """This method performs a gradient descent type optimization."""        
        for i in range(N):
            # 梯度清0
            optimizer.zero_grad()
            # 计算loss          
            loss,loss_valid = self.loss_fn(data[0], data[1],data[2],data[3])
            # 反向传播计算梯度
            loss.backward()
            # 根据loss调度学习率
            scheduler.step(loss)
            # 反向传播更新权重和偏置
            optimizer.step()

            # 记录loss并计算相对误差
            self.current_loss = loss.item()
            self.valid_loss=loss_valid.item()
           
            self.max_relative_error = self.ES()
            self.callback(self.max_relative_error,N)  # Pass max_relative_error to the callback function
            self.update_last_n_losses(self.current_loss)

            # 早停机制
            if self.max_relative_error < 2e-3: # in %
                print('Early stopping... \nIt {:05,d}: Loss = {:10.4e}, Max. rel. error = {} %'.format(self.iter,
                                                             self.current_loss,
                                                            np.round(self.max_relative_error, 3)))
                break

    # 打印loss    
    def callback(self, xr=None,N=1001):
        if self.iter % 100 == 0:
            print('It {:05,d}: Loss = {:10.4e}, Max. rel. error = {} %'.format(self.iter,
                                                             self.current_loss,
                                                            np.round(self.max_relative_error, 2)))
        self.hist[0].append(self.current_loss)
        self.hist[1].append(self.valid_loss)
        self.iter+=1
    
    def plot_loss_history(self, ax=None):
        if not ax:
            fig = plt.figure(figsize=(7, 5))
            ax = fig.add_subplot(111)

        # 绘制训练集损失曲线
        ax.semilogy(range(len(self.hist[0])), self.hist[0], 'b-', label='Training Loss')
        
        # 绘制验证集损失曲线
        ax.semilogy(range(len(self.hist[1])), self.hist[1], 'g-', label='Validation Loss')

        ax.set_xlabel('$n_{epoch}$')
        ax.set_ylabel('$loss$')
        ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, pos: f'{int(x):,}'))
        ax.legend()  # 添加图例
        return ax

In [46]:
model_HF_l = PINN_NeuralNet(input_dim=in_dim,
                             output_dim=out_dim,
                             num_hidden_layers=1,
                             num_neurons_per_layer=10,
                             activation='linear')
model_HF_nl = PINN_NeuralNet(input_dim=in_dim,
                             output_dim=out_dim,
                             num_hidden_layers=5,
                             num_neurons_per_layer=20,
                             activation='relu')
# 初始化 PINNSolver
solver = PINNSolver(model_HF_nl,model_HF_l)

model_HF_l.to(device)
model_HF_nl.to(device)
X_data_HF=X_data_HF.to(device)
y_data_HF=y_data_HF.to(device)
X_data_valid= X_data_valid.to(device)
y_data_valid=y_data_valid.to(device)

In [ ]:
# 定义学习率调度器
lr = 1e-4
optimizer = optim.Adam(
    list(model_HF_nl.parameters()) + list(model_HF_l.parameters()),  # 将生成器转换为列表并拼接
    lr=lr
)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.9, patience=10, verbose=True)
# 定义训练模式
mode = 'PyTorch_optimizer'
N = int(3000) + 1  # 训练迭代次数

try:
    runtime
except NameError:
    runtime = 0.

if mode == 'PyTorch_optimizer':
    try:
        t0 = time()
        solver.solve_with_PyTorch_optimizer(optimizer, [X_data_HF,y_data_HF,X_data_valid,y_data_valid],scheduler,N=N)
        runtime += (time() - t0) / 60.
        print('\nRuntime: {:.3f} minutes'.format(runtime))
    except KeyboardInterrupt:
        runtime += (time() - t0) / 60.
        print('\nRuntime: {:.3f} minutes'.format(runtime))

In [ ]:
solver.plot_loss_history()

In [ ]:
# 设置绘图样式
plt.rc('font', size=18)
plt.rc('axes', titlesize=18)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=18)
plt.rc('ytick', labelsize=18)
plt.rc('legend', fontsize=18)
plt.rc('figure', titlesize=18)
colors = ['tab:red', 'tab:orange', '#f9c74f', 'tab:green', 'tab:cyan', 'tab:blue', 'tab:purple', 'tab:brown', 'tab:pink']

# 将模型设置为评估模式
model_HF_nl.eval()
model_HF_l.eval()
# 创建图表
fig, ax = plt.subplots(figsize=(5, 5), dpi=300)

# 对训练集进行预测并绘制
for i, sheet_name in enumerate(df_HF.keys()):
    if sheet_name == 'b2' or sheet_name=='s' or sheet_name=='b1':
        continue  # 跳过验证集
    
    # 提取当前 sheet 的数据
    df = df_HF[sheet_name].dropna()
    y1_train = df['lossF'].to_numpy()
    X_train = df[feature_columns].to_numpy()
    
    # 对频率特征进行对数化处理
    X_train[:, 0] = np.log10(X_train[:, 0])
    X_train=add_hadamard_features(X_train)
    # 对当前 sheet 的数据进行归一化
    X_train_normalized = x_scaler.transform(X_train)
    y_train_normalized = y_scaler.transform(y1_train.reshape(-1, 1))
    
    # 将数据转换为 PyTorch 张量
    X_data_train = torch.tensor(X_train_normalized, dtype=torch.float32).to(device)
    # 模型预测
    y_MF_train = model_HF_nl(X_data_train)+model_HF_l(X_data_train)
    y_MF_train_denorm = y_scaler.inverse_transform(y_MF_train.detach().cpu().numpy())
    
    # 绘制当前 sheet 的拟合曲线
    ax.plot(10**X_train[:, 0], y_MF_train_denorm, color=colors[i], label=f'Train {sheet_name}')
    ax.scatter(10**X_train[:, 0], y1_train, color=colors[i], label=f'Train {sheet_name}')
# 对验证集进行预测并绘制
y_MF_valid = model_HF_nl(X_data_valid)+model_HF_l(X_data_valid)
y_MF_valid_denorm = y_scaler.inverse_transform(y_MF_valid.detach().cpu().numpy())

# 绘制验证集的拟合曲线
ax.plot(10**X_valid[:, 0], y_MF_valid_denorm, color='black', label='Validation Fit')

# 绘制验证集的真实值（空心圆）
y1_valid_denorm = y_scaler.inverse_transform(y_data_valid.cpu().numpy())
ax.scatter(10**X_valid[:, 0], y1_valid_denorm, color='black', marker='o', facecolors='none', s=50, label='Validation Data')

# 设置图表属性
ax.set_xscale('log')
ax.set_ylabel('$tan\delta$')
ax.set_xlabel('$\omega$ $\mathrm{[rad/s]}$')
plt.show()



# 计算验证集的 MAE、R2 和 MAPE
mae_valid = mean_absolute_error(y1_valid_denorm, y_MF_valid_denorm)
r2_valid = r2_score(y1_valid_denorm, y_MF_valid_denorm)
mape_valid = np.mean(np.abs((y1_valid_denorm - y_MF_valid_denorm) / y1_valid_denorm)) * 100

# 打印验证集的指标
print(f"Validation MAE: {mae_valid:.4f}")
print(f"Validation R2: {r2_valid:.4f}")
print(f"Validation MAPE: {mape_valid:.4f}%")

In [10]:
#torch.save(model_HF_l,'model/PFGs/model_l_AFF_pfgs')
#torch.save(model_HF_nl,'model/PFGs/model_nl_AFF_pfgs')


- 绘制各个模型的比对图

In [ ]:
# 设置绘图样式
plt.rc('font', size=18)
plt.rc('axes', titlesize=18)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=18)
plt.rc('ytick', labelsize=18)
plt.rc('legend', fontsize=18)
plt.rc('figure', titlesize=18)

# 加载模型
model_dnn = [torch.load('model/PFGs/model_l_dnn_pfgs'), torch.load('model/PFGs/model_nl_dnn_pfgs')]
model_pinn = [torch.load('model/PFGs/model_l_pinn_pfgs'), torch.load('model/PFGs/model_nl_pinn_pfgs')]
model_hadamard = [torch.load('model/PFGs/model_l_Hadamard_pfgs'), torch.load('model/PFGs/model_nl_Hadamard_pfgs')]
model_AFF = [torch.load('model/PFGs/model_l_AFF_pfgs'), torch.load('model/PFGs/model_nl_AFF_pfgs')]

# 将模型设置为评估模式
for model in [model_dnn, model_pinn, model_hadamard, model_AFF]:
    model[0].eval()
    model[1].eval()

# 创建图表
fig, ax = plt.subplots(figsize=(10, 8), dpi=300)

# 定义模型名称和颜色
model_names = ['DNN', 'PINN', 'PINN-HP', 'PINN-AFF']
model_colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:purple']

# 对验证集进行预测并绘制
for model, name, color in zip([model_dnn, model_pinn, model_hadamard, model_AFF], model_names, model_colors):
    # 模型预测
    y_MF_valid = model[1](X_data_valid) + model[0](X_data_valid)
    y_MF_valid_denorm = y_scaler.inverse_transform(y_MF_valid.detach().cpu().numpy())
    
    # 绘制验证集的拟合曲线
    ax.plot(10**X_valid[:, 0], y_MF_valid_denorm, color=color, label=f'{name}', linewidth=3)  # 设置曲线宽度
    
    # 计算验证集的 MAE、R2 和 MAPE
    mae_valid = mean_absolute_error(y1_valid_denorm, y_MF_valid_denorm)
    r2_valid = r2_score(y1_valid_denorm, y_MF_valid_denorm)
    mape_valid = np.mean(np.abs((y1_valid_denorm - y_MF_valid_denorm) / y1_valid_denorm)) * 100
    
    # 打印验证集的指标
    print(f"{name} Validation MAE: {mae_valid:.4f}")
    print(f"{name} Validation R2: {r2_valid:.4f}")
    print(f"{name} Validation MAPE: {mape_valid:.4f}%")
    print("-" * 40)

# 绘制验证集的真实值（空心圆）
y1_valid_denorm = y_scaler.inverse_transform(y_data_valid.cpu().numpy())
ax.scatter(10**X_valid[:, 0], y1_valid_denorm, color='black', marker='o', facecolors='none', s=100, label='Test')  # 设置点图大小

# 设置图表属性
ax.set_xscale('log')
ax.set_ylabel('$tan\delta$', fontsize=24)  # 设置纵坐标名称字体大小
ax.set_xlabel('$\omega$ $\mathrm{[rad/s]}$', fontsize=24)  # 设置横坐标名称字体大小
ax.tick_params(axis='both', which='major', labelsize=24)  # 设置横纵坐标刻度字体大小
ax.legend(loc='upper right', bbox_to_anchor=(0.35, 1), fontsize=20)  # 设置图例位置和大小
plt.tight_layout()
plt.show()

- 绘制残差图

In [ ]:
# 设置绘图样式
plt.rc('font', size=18)
plt.rc('axes', titlesize=18)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=18)
plt.rc('ytick', labelsize=18)
plt.rc('legend', fontsize=18)
plt.rc('figure', titlesize=18)

# 加载模型
model_dnn = [torch.load('model/PFGs/model_l_dnn_pfgs'), torch.load('model/PFGs/model_nl_dnn_pfgs')]
model_pinn = [torch.load('model/PFGs/model_l_pinn_pfgs'), torch.load('model/PFGs/model_nl_pinn_pfgs')]
model_hadamard = [torch.load('model/PFGs/model_l_Hadamard_pfgs'), torch.load('model/PFGs/model_nl_Hadamard_pfgs')]
model_AFF = [torch.load('model/PFGs/model_l_AFF_pfgs'), torch.load('model/PFGs/model_nl_AFF_pfgs')]

# 将模型设置为评估模式
for model in [model_dnn, model_pinn, model_hadamard, model_AFF]:
    model[0].eval()
    model[1].eval()

# 创建残差图
fig, ax = plt.subplots(figsize=(10, 8), dpi=300)

# 定义模型名称和颜色
model_names = ['DNN', 'PINN', 'PINN-HP', 'PINN-AFF']
model_colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:purple']

# 对验证集进行预测并绘制残差图
for model, name, color in zip([model_dnn, model_pinn, model_hadamard, model_AFF], model_names, model_colors):
    # 模型预测
    y_MF_valid = model[1](X_data_valid) + model[0](X_data_valid)
    y_MF_valid_denorm = y_scaler.inverse_transform(y_MF_valid.detach().cpu().numpy())
    
    # 计算残差
    residuals = y1_valid_denorm - y_MF_valid_denorm
    
    # 绘制残差图
    
    ax.scatter(X_valid[:, 0], residuals, color=color, label=f'{name}', s=50)  # 设置点图大小

# 绘制 y=0 参考线
ax.axhline(y=0, color='black', linestyle='--', linewidth=2)

# 设置图表属性
ax.set_xlabel('$lg\omega$', fontsize=24)  # 设置横坐标名称字体大小
ax.set_ylabel('Residuals', fontsize=24)  # 设置纵坐标名称字体大小
ax.tick_params(axis='both', which='major', labelsize=24)  # 设置横纵坐标刻度字体大小
ax.legend(loc='upper right', bbox_to_anchor=(0.3, 1), fontsize=18)  # 设置图例位置和大小
ax.set_ylim(-0.4,0.4)
plt.tight_layout()
plt.show()

## 绘制指标图（柱状图）

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

# 示例数据
metrics = [f'$R^{2}$', 'MAE', 'MAPE', 'Training Time']
dnn_values = [0.7559, 0.1080, 0.1909, 499]  # DNN 值
pinn_values = [0.8152, 0.1003, 0.1501, 546]  # PINN 值
pinn_hp_values = [0.8760, 0.0803, 0.1323, 508]  # PINN-HP 值
pinn_aff_values = [0.9873, 0.0244, 0.0324, 522]  # PINN-AFF 值

# 误差棒（请根据实际情况替换）
dnn_errors = [0.008, 0.0008, 0.0008, 5]  # DNN 误差
pinn_errors = [0.008, 0.0003, 0.0005, 4]  # PINN 误差
pinn_hp_errors = [0.007, 0.0003, 0.0005, 3]  # PINN-HP 误差
pinn_aff_errors = [0.006, 0.0003, 0.0005, 4]  # PINN-AFF 误差

# 颜色设置
dnn_color = 'tab:blue'  # 蓝色
pinn_color = 'tab:orange'  # 橙色
pinn_hp_color = 'tab:green'  # 绿色
pinn_aff_color = 'tab:purple'  # 紫色

# 创建渐变颜色映射
def create_gradient_cmap(color1, color2, gamma=2.0, N=256):
    cmap = LinearSegmentedColormap.from_list('custom_cmap', [color1, color2], N=N)
    cmap._init()
    cmap._lut[:, -1] = np.linspace(0, 1, cmap.N + 3) ** gamma
    return cmap

# 绘制柱状图
def plot_bar(ax, x, value, error, color, label):
    bar = ax.bar(x, value, width=0.1, color=color, yerr=error, capsize=14, label=label)
    for rect in bar:
        height = rect.get_height()
        # 标签的 y 坐标 = 柱子高度 + 误差棒高度
        label_y = height + error  # 使用当前指标的误差值
        if metric == 'Training Time':
            ax.text(rect.get_x() + rect.get_width() / 2.0, label_y, f'{height:.0f}', ha='center', va='bottom', fontsize=18)
        else:
            ax.text(rect.get_x() + rect.get_width() / 2.0, label_y, f'{height:.4f}', ha='center', va='bottom', fontsize=18)

# 绘图
for i, metric in enumerate(metrics):
    fig, ax = plt.subplots(figsize=(10, 8), dpi=300)
    x = [1, 1.2, 1.4, 1.6]  # 柱状图位置：DNN, PINN, PINN-HP, PINN-AFF
    
    # 创建渐变颜色
    dnn_cmap = create_gradient_cmap('lightblue', dnn_color)
    pinn_cmap = create_gradient_cmap('white', pinn_color)
    pinn_hp_cmap = create_gradient_cmap('white', pinn_hp_color)
    pinn_aff_cmap = create_gradient_cmap('white', pinn_aff_color)
    
    # 绘制柱状图
    plot_bar(ax, x[0], dnn_values[i], dnn_errors[i], dnn_cmap(0.8), 'DNN')
    plot_bar(ax, x[1], pinn_values[i], pinn_errors[i], pinn_cmap(0.8), 'PINN')
    plot_bar(ax, x[2], pinn_hp_values[i], pinn_hp_errors[i], pinn_hp_cmap(0.8), 'PINN-HP')
    plot_bar(ax, x[3], pinn_aff_values[i], pinn_aff_errors[i], pinn_aff_cmap(0.8), 'PINN-AFF')
    
    # 设置坐标轴
    ax.set_xticks(x)
    ax.set_xticklabels(['DNN', 'PINN', 'PINN-HP', 'PINN-AFF'], fontsize=24)
    ax.set_ylabel(metric + (' [s]' if metric == 'Training Time' else ''), fontsize=24)
    ax.set_ylim(0, max(dnn_values[i], pinn_values[i], pinn_hp_values[i], pinn_aff_values[i]) * 1.2)
    ax.tick_params(axis='y', labelsize=24)  # 设置 y 轴刻度字体大小为 24
    
    plt.tight_layout()
    plt.show()